# Tutorial: Separator Unit Model with Modular Property Package

![](splitter.svg)

## Learning Outcomes

- Demonstrate use of the separator unit model in IDAES
- Demonstrate different separation options available


## Problem Statement

In this example, we will be separating a feed stream of liquid H2O and CO2 vapor. The inlet conditions are as follows:

**Inlet:**

Flow Rate = 100 mol/s

Mole Fraction (H2O) = 0.9

Mole Fraction (CO2) = 0.1

Temperature = 298 K

Pressure = 101325 Pa

We will look at three cases in this tutorial:

Case 1: Separate on a total flow split basis

Case 2: Separate on a phase flow split basis

Case 3: Separate on a component flow split basis


For more details, please refer to the IDAES documentation: https://idaes-pse.readthedocs.io/en/stable

## Importing necessary tools

In the following cell, we will be importing the necessary components from Pyomo and IDAES.

In [ ]:
# Import objects from pyomo package 
from pyomo.environ import ConcreteModel, value

# Import the solver
from idaes.core.solvers import get_solver

# Import the main FlowsheetBlock from IDAES. The flowsheet block will contain the unit model
from idaes.core import FlowsheetBlock

# Import the option to set the type of material balance 
from idaes.core import MaterialBalanceType

# Import the separator unit model
from idaes.models.unit_models import Separator

# Import the option to set the basis for splitting
from idaes.models.unit_models.separator import SplittingType

# Import idaes logger to set output levels
import idaes.logger as idaeslog

# Import the modular property package to create a property block for the flowsheet
from idaes.models.properties.modular_properties.base.generic_property import GenericParameterBlock

# Import the BT_Ideal property package to create a configuration file for the GenericParameterBlock
from idaes.models.properties.modular_properties.examples.CO2_H2O_Ideal_VLE import configuration

# Import the degrees_of_freedom function from the idaes.core.util.model_statistics package
# DOF = Number of Model Variables - Number of Model Constraints
from idaes.core.util.model_statistics import degrees_of_freedom

## Setting up the flowsheet

In the following cell, we will create the `ConcreteModel` foundation, attach the steady state flowsheet, and declare the property parameter block that will used.

More information on this general workflow can be found here: https://idaes-pse.readthedocs.io/en/stable/how_to_guides/workflow/general.html

In [ ]:
m = ConcreteModel()

m.fs = FlowsheetBlock(dynamic=False) # dynamic or ss flowsheet needs to be specified here

m.fs.properties = GenericParameterBlock(**configuration)

## Case 1:

In the following cell, we will be creating the separator unit model that splits on a *total flow basis* and determining the initial degrees of freedom associated with this separator unit model.

In [ ]:
m.fs.sep_1 = Separator(
    property_package=m.fs.properties,
    split_basis=SplittingType.totalFlow,
    outlet_list=["a1", "b1", "c1"],  # creates three outlet streams
    ideal_separation=False,
    has_phase_equilibrium=False,
)

DOF_initial = degrees_of_freedom(m)
print('The initial degrees of freedom are: {0}'.format(DOF_initial))

### Fixing input specifications
In the following cell, we will be specifying the inlet conditions for the separator block and re-evaluating the degrees of freedom to ensure the problem is square (i.e. DOF=0).

In [ ]:
m.fs.sep_1.inlet.flow_mol.fix(10) # converting to mol/s as unit basis is mol/s
m.fs.sep_1.inlet.mole_frac_comp[0, "H2O"].fix(0.9)
m.fs.sep_1.inlet.mole_frac_comp[0, "CO2"].fix(0.1)
m.fs.sep_1.inlet.pressure.fix(101325) # Pa
m.fs.sep_1.inlet.temperature.fix(353) # K

m.fs.sep_1.split_fraction[0, "a1"].fix(0.2)
m.fs.sep_1.split_fraction[0, "b1"].fix(0.5)
# Directly setting the split fraction of c1 will cause the DOF check to fail

DOF_final = degrees_of_freedom(m)
print('The final degrees of freedom is: {0}'.format(DOF_final))

### Flowsheet Initialization

IDAES includes pre-written initialization routines for all unit models. Failing to initialize or having a poor intialization of a flowsheet may result in the problem being unsolvable. The output from initialization can be set to 7 different levels depending on the details required by the user. In general, when a particular output level is set, any information at that level and above gets picked up by logger. The default level taken by the logger is INFO.

More information on these levels can be found in the IDAES documentation: https://idaes-pse.readthedocs.io/en/stable/reference_guides/logging.html

In [ ]:
m.fs.sep_1.initialize(outlvl=idaeslog.WARNING)

### Obtaining Simulation Results

In the following cell, the flowsheet will be solved using the IDAES `get_solver` tool. Setting `tee=True` will display the solver output.

In [ ]:
solver = get_solver()
result = solver.solve(m, tee=True)

### View Results
As expected, the report below shows that the three outlet streams share the same composition as the inlet stream with the only difference being their molar flowrate, which was determined by their respective split fractions.

In [ ]:
m.fs.sep_1.report()

## Case 2:

In the following cell, we will be creating the separator unit model that splits on a *phase flow basis* and determining the initial degrees of freedom associated with this separator unit model.

In [ ]:
m.fs.sep_2 = Separator(
    property_package=m.fs.properties,
    split_basis=SplittingType.phaseFlow,
    outlet_list=["a2", "b2"],
    ideal_separation=False,
    has_phase_equilibrium=False,
)

DOF_initial = degrees_of_freedom(m)
print('The initial degrees of freedom are: {0}'.format(DOF_initial))

### Fixing input specifications
In the following cell, we will be specifying the inlet conditions for the separator block and re-evaluating the degrees of freedom to ensure the problem is square (i.e. DOF=0).

In [ ]:
m.fs.sep_2.inlet.flow_mol.fix(10) # converting to mol/s as unit basis is mol/s
m.fs.sep_2.inlet.mole_frac_comp[0, "H2O"].fix(0.9)
m.fs.sep_2.inlet.mole_frac_comp[0, "CO2"].fix(0.1)
m.fs.sep_2.inlet.pressure.fix(101325) # Pa
m.fs.sep_2.inlet.temperature.fix(353) # K

m.fs.sep_2.split_fraction[0, "a2", "Vap"].fix(0.8)
m.fs.sep_2.split_fraction[0, "b2", "Liq"].fix(0.8)

DOF_final = degrees_of_freedom(m)
print('The final degrees of freedom is: {0}'.format(DOF_final))

### Flowsheet Initialization

IDAES includes pre-written initialization routines for all unit models. Failing to initialize or having a poor intialization of a flowsheet may result in the problem being unsolvable. The output from initialization can be set to 7 different levels depending on the details required by the user. In general, when a particular output level is set, any information at that level and above gets picked up by logger. The default level taken by the logger is INFO.

More information on these levels can be found in the IDAES documentation: https://idaes-pse.readthedocs.io/en/stable/reference_guides/logging.html

In [ ]:
m.fs.sep_2.initialize(outlvl=idaeslog.WARNING)

### Obtaining Simulation Results

In the following cell, the flowsheet will be solved using the IDAES `get_solver` tool. Setting `tee=True` will display the solver output.

In [ ]:
solver = get_solver()
result = solver.solve(m, tee=True)

### View Results
As expected, the report below shows that the outlet *a2* contains 80% of the gaseous CO2 and outlet *b2* contains 80% of the liquid H2O.

In [ ]:
m.fs.sep_2.report()

## Case 3:

In the following cell, we will be creating the separator unit model that splits on a *component flow basis* and determining the initial degrees of freedom associated with this separator unit model.

In [ ]:
m.fs.sep_3 = Separator(
    property_package=m.fs.properties,
    split_basis=SplittingType.componentFlow,
    outlet_list=["a3", "b3", "c3", "d3"],
    ideal_separation=False,
    has_phase_equilibrium=False,
)

DOF_initial = degrees_of_freedom(m)
print('The initial degrees of freedom are: {0}'.format(DOF_initial))

### Fixing input specifications
In the following cell, we will be specifying the inlet conditions for the separator block and re-evaluating the degrees of freedom to ensure the problem is square (i.e. DOF=0).

In [ ]:
m.fs.sep_3.inlet.flow_mol.fix(10) # converting to mol/s as unit basis is mol/s
m.fs.sep_3.inlet.mole_frac_comp[0, "H2O"].fix(0.9)
m.fs.sep_3.inlet.mole_frac_comp[0, "CO2"].fix(0.1)
m.fs.sep_3.inlet.pressure.fix(101325) # Pa
m.fs.sep_3.inlet.temperature.fix(353) # K

m.fs.sep_3.split_fraction[0, "a3", "H2O"].fix(0.25)
m.fs.sep_3.split_fraction[0, "b3", "H2O"].fix(0.5)
m.fs.sep_3.split_fraction[0, "c3", "H2O"].fix(0.1)

m.fs.sep_3.split_fraction[0, "a3", "CO2"].fix(0.1)
m.fs.sep_3.split_fraction[0, "b3", "CO2"].fix(0.2)
m.fs.sep_3.split_fraction[0, "c3", "CO2"].fix(0.3)

DOF_final = degrees_of_freedom(m)
print('The final degrees of freedom is: {0}'.format(DOF_final))

### Flowsheet Initialization

IDAES includes pre-written initialization routines for all unit models. Failing to initialize or having a poor intialization of a flowsheet may result in the problem being unsolvable. The output from initialization can be set to 7 different levels depending on the details required by the user. In general, when a particular output level is set, any information at that level and above gets picked up by logger. The default level taken by the logger is INFO.

More information on these levels can be found in the IDAES documentation: https://idaes-pse.readthedocs.io/en/stable/reference_guides/logging.html

In [ ]:
m.fs.sep_3.initialize(outlvl=idaeslog.WARNING)

### Obtaining Simulation Results

In the following cell, the flowsheet will be solved using the IDAES `get_solver` tool. Setting `tee=True` will display the solver output.

In [ ]:
solver = get_solver()
result = solver.solve(m, tee=True)

### View Results
As expected, the report below shows that the component mole fractions in each of the outlet streams are equivalent to their respective split fractions as specified in the input specifications.

In [ ]:
m.fs.sep_3.report()